In [ ]:
!pip install -q fuzzywuzzy python-Levenshtein Levenshtein nltk pandas pyarrow seaborn

In [ ]:
import pandas as pd
import seaborn as sns
from Levenshtein import distance
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from nltk import edit_distance
import matplotlib.pyplot as plt
import numpy as np
import os

def compute_Levensthein_sim(str1,str2):
    edit_dist  = edit_distance(str1,str2)
    len_total = max(len(str1),len(str2))
    return ((len_total-edit_dist)/len_total)*100

def match_part(x,index):
    return x.str.split("-----")[0][index-1] if len(x.str.split("-----")[0])>=index  else None



In [ ]:
PATH_TO_FILE="../../resultats_stage/"
FILENAME_INPUT_RESULTS="Dares_1Kaccords_dataset_txt_label_result.parquet"

In [ ]:
df_results=pd.read_parquet(os.path.join(PATH_TO_FILE,FILENAME_INPUT_RESULTS))

In [ ]:
df_results["titre_label"]=df_results[['label']].apply(lambda x : match_part(x,1),axis=1)
df_results["corps_label"]=df_results[['label']].apply(lambda x : match_part(x,2) ,axis=1)
df_results["annexe_label"]=df_results[['label']].apply(lambda x : match_part(x,3),axis=1)
df_results["titre_result_1a"]=df_results[['result_1a']].apply(lambda x : match_part(x,1),axis=1)
df_results["corps_result_1a"]=df_results[['result_1a']].apply(lambda x : match_part(x,2) ,axis=1)
df_results["titre_result_1b"]=df_results[['result_1b']].apply(lambda x : match_part(x,1),axis=1)
df_results["corps_result_1b"]=df_results[['result_1b']].apply(lambda x : match_part(x,2) ,axis=1)
df_results["titre_1a_Levensthein"]=df_results[["titre_label","titre_result_1a"]].apply(lambda x :distance(*x),axis=1)
df_results["titre_1a_Levensthein_sim"]=df_results[["titre_label","titre_result_1a"]].apply(lambda x :compute_Levensthein_sim(*x),axis=1)
df_results["titre_1a_fuzz_ratio"]=df_results[["titre_label","titre_result_1a"]].apply(lambda x :fuzz.ratio(*x),axis=1)
df_results["titre_1a_fuzz_partial_ratio"]=df_results[["titre_label","titre_result_1a"]].apply(lambda x :fuzz.partial_ratio(*x),axis=1)
df_results["titre_1b_Levensthein"]=df_results[["titre_label","titre_result_1b"]].apply(lambda x :distance(*x),axis=1)
df_results["titre_1b_Levensthein_sim"]=df_results[["titre_label","titre_result_1b"]].apply(lambda x :compute_Levensthein_sim(*x),axis=1)
df_results["titre_1b_fuzz_ratio"]=df_results[["titre_label","titre_result_1b"]].apply(lambda x :fuzz.ratio(*x),axis=1)
df_results["titre_1b_fuzz_partial_ratio"]=df_results[["titre_label","titre_result_1b"]].apply(lambda x :fuzz.partial_ratio(*x),axis=1)

In [ ]:
sns.displot(df_results[["titre_1a_fuzz_partial_ratio","titre_1a_fuzz_ratio","titre_1a_Levensthein_sim"]],multiple="dodge",kde=True,bins=10)

In [ ]:
sns.displot(df_results[["titre_1a_fuzz_partial_ratio","titre_1b_fuzz_partial_ratio"]],multiple="dodge",kde=True,bins=10)

In [ ]:
sns.displot(df_results[["titre_1a_fuzz_ratio","titre_1b_fuzz_ratio"]],multiple="dodge",kde=True,bins=10)

In [ ]:
sns.displot(df_results[["titre_1a_Levensthein_sim","titre_1b_Levensthein_sim"]],multiple="dodge",kde=True,bins=10)

In [ ]:
df_results[df_results["titre_1a_Levensthein_sim"].between(31,40)][["titre_label","titre_result_1a","result_1a","titre_1a_Levensthein_sim","titre_1a_fuzz_ratio","titre_result_1b","result_1b","titre_1b_Levensthein_sim"]]           

In [ ]:
df_results["titre_1a_Levensthein_sim_cat"]=pd.cut(df_results["titre_1a_Levensthein_sim"],[0,20,50,80,100])
CATEGORIES = ['Mauvais','Passable','Correct','Excellent']
df_results["titre_1a_Levensthein_sim_cat"]=df_results.titre_1a_Levensthein_sim_cat.cat.rename_categories(CATEGORIES)
df_results["titre_1b_Levensthein_sim_cat"]=pd.cut(df_results["titre_1b_Levensthein_sim"],[0,20,50,80,100])
df_results["titre_1b_Levensthein_sim_cat"]=df_results.titre_1b_Levensthein_sim_cat.cat.rename_categories(CATEGORIES)
df_results["titre_1a_fuzz_ratio_cat"]=pd.cut(df_results["titre_1a_fuzz_ratio"],[0,20,50,80,100])
df_results["titre_1a_fuzz_ratio_cat"]=df_results.titre_1a_fuzz_ratio_cat.cat.rename_categories(CATEGORIES)
df_results["titre_1b_fuzz_ratio_cat"]=pd.cut(df_results["titre_1b_fuzz_ratio"],[0,20,50,80,100])
df_results["titre_1b_fuzz_ratio_cat"]=df_results.titre_1b_fuzz_ratio_cat.cat.rename_categories(CATEGORIES)
df_results["titre_1a_fuzz_partial_ratio_cat"]=pd.cut(df_results["titre_1a_fuzz_partial_ratio"],[0,20,50,80,100])
df_results["titre_1a_fuzz_partial_ratio_cat"]=df_results.titre_1a_fuzz_partial_ratio_cat.cat.rename_categories(CATEGORIES)
df_results["titre_1b_fuzz_partial_ratio_cat"]=pd.cut(df_results["titre_1b_fuzz_partial_ratio"],[0,20,50,80,100])
df_results["titre_1b_fuzz_partial_ratio_cat"]=df_results.titre_1b_fuzz_partial_ratio_cat.cat.rename_categories(CATEGORIES)

In [ ]:
fig, ax =plt.subplots(1,3,figsize=(10, 5))
sns.countplot(x=df_results["titre_1a_Levensthein_sim_cat"],ax=ax[0])
sns.countplot(x=df_results["titre_1a_fuzz_ratio_cat"],ax=ax[1])
sns.countplot(x=df_results["titre_1a_fuzz_partial_ratio_cat"],ax=ax[2])
fig.tight_layout(pad=0.5)


fig.show()

In [ ]:
df_results[df_results["titre_1a_fuzz_partial_ratio_cat"]=="Excellent"].shape

In [ ]:
df_results[df_results["titre_1b_fuzz_partial_ratio_cat"]=="Excellent"].shape

In [ ]:
pd.crosstab( df_results.titre_1a_fuzz_ratio_cat,df_results.titre_1a_fuzz_partial_ratio_cat)

In [ ]:
pd.crosstab( df_results.titre_1a_Levensthein_sim_cat,df_results.titre_1a_fuzz_partial_ratio_cat)

In [ ]:
pd.crosstab( df_results.titre_1a_Levensthein_sim_cat,df_results.titre_1a_fuzz_ratio_cat)

In [ ]:
pd.crosstab( df_results.titre_1a_Levensthein_sim_cat,df_results.titre_1b_Levensthein_sim_cat)

In [ ]:
matrix=pd.crosstab( df_results.titre_1a_fuzz_ratio_cat,df_results.titre_1b_fuzz_ratio_cat)

In [ ]:
def calculate_sum_over_under_diag_over_matrix(matrix):
    rng = np.arange(len(matrix))
    ut=matrix.mask(rng[:, None] <= rng).reset_index(drop=True).sum().sum()
    diag=np.diag(matrix).sum()
    ot=matrix.mask(rng[:, None] >= rng).reset_index(drop=True).sum().sum()
    return (ut,diag,ot)

In [ ]:
list(map(lambda x: x/10,calculate_sum_over_under_diag_over_matrix(matrix)))

In [ ]:
fig, ax =plt.subplots(1,3,figsize=(10, 5))
sns.countplot(x=df_results["titre_1b_Levensthein_sim_cat"],ax=ax[0])
sns.countplot(x=df_results["titre_1b_fuzz_ratio_cat"],ax=ax[1])
sns.countplot(x=df_results["titre_1b_fuzz_partial_ratio_cat"],ax=ax[2])
fig.tight_layout(pad=0.5)
fig.show()

In [ ]:
df_results.corps_label

In [ ]:

df_results["corps_result_1a"]

In [ ]:
def compare_deux_chaines_corps_compte_manquant_label(label,resultat):
    manquant=0
    total_item=0
    for item in label.split("\n"):
        if item:
            total_item+=1
            if max([fuzz.partial_ratio(item,item_resultat) for item_resultat in resultat.split("\n")])<80:
                manquant+=1
    return (manquant,total_item)

In [ ]:
df_results[["corps_manquant_1a","total_item"]]=df_results[["corps_label","corps_result_1a"]].apply(lambda x: compare_deux_chaines_corps_compte_manquant_label(*x),axis=1, result_type="expand")
df_results[["corps_manquant_1b","total_item"]]=df_results[["corps_label","corps_result_1b"]].apply(lambda x: compare_deux_chaines_corps_compte_manquant_label(*x),axis=1, result_type="expand")

In [ ]:
df_results["corps_manquant_1a_pct"]= df_results.corps_manquant_1a*100/df_results.total_item
df_results["corps_manquant_1b_pct"]= df_results.corps_manquant_1b*100/df_results.total_item

In [ ]:
sns.displot(df_results[["corps_manquant_1a_pct","corps_manquant_1b_pct"]],kde=True)

In [ ]:
df_results["total_item"]

In [ ]:
sns.displot(df_results[df_results["total_item"]<40][["corps_manquant_1a"]])

In [ ]:
df_results["result_2_string"]=df_results[["result_2"]].apply(lambda x: "\n".join(x.values[0]),axis=1)

In [ ]:
df_results[["corps_manquant_2","total_item"]]=df_results[["corps_label","result_2_string"]].apply(lambda x: compare_deux_chaines_corps_compte_manquant_label(*x),axis=1, result_type="expand")
df_results["corps_manquant_2_pct"]= df_results.corps_manquant_2*100/df_results.total_item

In [ ]:
sns.displot(df_results[["corps_manquant_2_pct"]],kde=True)

In [ ]:
sns.displot(df_results[["corps_manquant_2"]],kde=True)